<a href="https://colab.research.google.com/github/1danjae1/Maritime-Data-Mining/blob/main/%ED%95%B4%EC%82%AC%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A7%88%EC%9D%B4%EB%8B%9D%2013%EC%A3%BC%EC%B0%A8%20%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
data = pd.read_csv('./seoul-metro-2021.logs.csv')
data

,timestamp,station_code,people_in,people_out
0,2021-01-01T05:00:00.000+09:00,150,86,85
1,2021-01-01T06:00:00.000+09:00,150,111,355
2,2021-01-01T07:00:00.000+09:00,150,157,438
3,2021-01-01T08:00:00.000+09:00,150,306,592
4,2021-01-01T09:00:00.000+09:00,150,333,841
...,...,...,...,...
1941548,2021-12-31T19:00:00.000+09:00,2828,74,263
1941549,2021-12-31T20:00:00.000+09:00,2828,73,145
1941550,2021-12-31T21:00:00.000+09:00,2828,95,209
1941551,2021-12-31T22:00:00.000+09:00,2828,54,138


In [2]:
# 1️⃣ 라이브러리 설치 및 불러오기
!pip install folium
import pandas as pd
import folium
from folium.plugins import HeatMap

# 2️⃣ 데이터 불러오기
# 지하철 시간대별 승하차 인원 데이터
log_data = pd.read_csv('/content/seoul-metro-2021.logs.csv')
# 지하철역 위치 정보 데이터
station_info = pd.read_csv('/content/seoul-metro-station-info.csv')

# 3️⃣ 데이터 전처리
# 시간 데이터를 datetime 형식으로 변환
log_data['timestamp'] = pd.to_datetime(log_data['timestamp'], errors='coerce')

# 출근(오전 9시 이전) 데이터 추출
morning_data = log_data[log_data['timestamp'].dt.hour < 9]
# 퇴근(오후 5시 이후) 데이터 추출
evening_data = log_data[log_data['timestamp'].dt.hour >= 17]

# 그룹화하여 역별 승하차 인원 합계 계산
morning_sum = morning_data.groupby('station_code')[['people_in', 'people_out']].sum()
evening_sum = evening_data.groupby('station_code')[['people_in', 'people_out']].sum()

# 지하철역 위치 정보에서 필요한 컬럼만 추출
station_info_filtered = station_info[['station.code', 'geo.latitude', 'geo.longitude']].rename(columns={'station.code': 'station_code'})
station_info_filtered = station_info_filtered.set_index('station_code')

# 데이터 병합
morning_joined = morning_sum.join(station_info_filtered)
evening_joined = evening_sum.join(station_info_filtered)

# 4️⃣ Folium 지도에 시각화
# 지도의 초기 위치(서울 중심부)
center_lat, center_lon = 37.55, 126.98

# 출근시간 승차 히트맵
map_morning_in = folium.Map(location=[center_lat, center_lon], zoom_start=12)
HeatMap(data=morning_joined[['geo.latitude', 'geo.longitude', 'people_in']].dropna().values, radius=10).add_to(map_morning_in)
map_morning_in.save('morning_in.html')

# 출근시간 하차 히트맵
map_morning_out = folium.Map(location=[center_lat, center_lon], zoom_start=12)
HeatMap(data=morning_joined[['geo.latitude', 'geo.longitude', 'people_out']].dropna().values, radius=10).add_to(map_morning_out)
map_morning_out.save('morning_out.html')

# 퇴근시간 승차 히트맵
map_evening_in = folium.Map(location=[center_lat, center_lon], zoom_start=12)
HeatMap(data=evening_joined[['geo.latitude', 'geo.longitude', 'people_in']].dropna().values, radius=10).add_to(map_evening_in)
map_evening_in.save('evening_in.html')

# 퇴근시간 하차 히트맵
map_evening_out = folium.Map(location=[center_lat, center_lon], zoom_start=12)
HeatMap(data=evening_joined[['geo.latitude', 'geo.longitude', 'people_out']].dropna().values, radius=10).add_to(map_evening_out)
map_evening_out.save('evening_out.html')

# 5️⃣ 분석 결과 도출
# 가장 유동인구 많은 역 3곳
print("출근시간 승차 인원 TOP3:")
print(morning_joined.sort_values('people_in', ascending=False).head(3))

print("\n출근시간 하차 인원 TOP3:")
print(morning_joined.sort_values('people_out', ascending=False).head(3))

print("\n퇴근시간 승차 인원 TOP3:")
print(evening_joined.sort_values('people_in', ascending=False).head(3))

print("\n퇴근시간 하차 인원 TOP3:")
print(evening_joined.sort_values('people_out', ascending=False).head(3))


출근시간 승차 인원 TOP3:
              people_in  people_out  geo.latitude  geo.longitude
station_code                                                    
230             6939358     1339845     37.484201     126.929715
232             4506454     4092652     37.485266     126.901401
228             4167880     1229054     37.481247     126.952739

출근시간 하차 인원 TOP3:
              people_in  people_out  geo.latitude  geo.longitude
station_code                                                    
2748             697336     6067855     37.481072     126.882343
222             1316542     5757499     37.497175     127.027926
221              546480     5521983     37.500622     127.036456

퇴근시간 승차 인원 TOP3:
              people_in  people_out  geo.latitude  geo.longitude
station_code                                                    
222            14845284     5809857     37.497175     127.027926
219             9535958     1988048     37.508844     127.063160
239             9453758     6443795  